In [ ]:
#@title 1- Para iniciar o programa, favor carregar as bibliotecas apertando o botão de  [  ▶ ] abaixo:

print("######################################################")
print("Instalando e carregando as bibliotecas necessárias...")
print("######################################################\n")

# Instalação de bibliotecas
! pip install pulp # lib de Lp (Linear Programing) para otimização dos parâmetros

# Importando as bibliotecas
import os
import pickle
import pandas as pd
from datetime import datetime
import pulp
from pulp import LpMinimize, LpProblem, LpStatus, lpSum, LpVariable, LpConstraint, LpMaximize

## Biblioteca para uso em VM no Google Collab
from google.colab import files
from google.colab import drive

# Removendo Warnings
import warnings
warnings.filterwarnings('ignore')

# Montando o diretório do Drive para leitura de arquivos do programa
try:
  drive.mount('/content/drive')
except:
  print("\nÉ necessário permitir que este notebook acesse seus arquivos do Google Drive.")
  print("Favor executar novamente e autorizar o acesso.")

else:
  print("\n\n##############################################################################")
  print("Bibliotecas carregadas com sucesso!")

  # Funções para predição da Expansão
  def load_model(modelName):
    """Carregamento do ML model"""
    model = pickle.load(open(modelName,'rb')) # Carrega modelo gerado
    return model

  def load_stdScaler(scName):
    """Carregamento do Fit para Standard Scaler"""
    model_sc_fit, linear_sc_fit = pickle.load(open(scName,'rb')) # Carrega StandardScale fit gerado
    return model_sc_fit, linear_sc_fit

  def load_file(file_name):
    """Carregamento e preparação dos dados para a utilização no modelo"""
    data = pd.read_excel(file_name, dtype={'Etapa':'string', 'Otimo':'float64', 'LIE':'float64', 'LSE':'float64'})
    return data

  def transform_model_data(file_name, model_sc_fit):
    """Pivotamento e transformação dos dados para o modelo de predição"""
    data = load_file(file_name)

    # Pivotamento dos dados carregados
    table = pd.pivot_table(data, values='Otimo', index=['id'], columns=['Etapa']).reset_index()
    
    # Organização das colunas conforme ordem definido durante a geração do modelo
    colunas = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 
              'quantidade_de_fermento', 'tempo_de_batimento_1ª_fase', 'tempo_de_batimento_2ª_fase', 'temperatura_de_massa',
              'peso_cru', 'altura_da_faca','pressão_rolo_de_borracha_direito', 'pressão_rolo_de_borracha_esquerdo','velocidade_da_moldadora',
              'velocidade_forçador', 'velocidade_lona_moldadora', 'pressão_rolo_estriado',
              'tempo_de_cozimento', 'pressão_do_lastro_-_zona_1', 'pressão_do_lastro_-_zona_2', 'pressão_do_lastro_-_zona_3', 'pressão_do_lastro_-_zona_4',
              'pressão_do_teto_-_zona_1', 'pressão_do_teto_-_zona_2', 'pressão_do_teto_-_zona_3', 'pressão_do_teto_-_zona_4',
              'temperatura_zona_1', 'temperatura_zona_2', 'temperatura_zona_3', 'temperatura_zona_4',
              'comprimento','largura', 'peso_assado', 'umidade_(água)', 'ph', 'cor_hunter_l_-_teto']
  
    table= table.loc[:, colunas]

    table = pd.DataFrame(model_sc_fit.transform(table), columns=colunas) # Aplicando a normalização aos dados inseridos
    return table

  def transform_lp_data(file_name, linear_sc_fit):
    """Pivotamento e transformação dos dados para o modelo de otimização do parâmetros"""
    data = load_file(file_name)

    # Pivotamento dos dados carregados
    table = pd.pivot_table(data, values=['Otimo', 'LIE', 'LSE'], index=['id'], columns=['Etapa']).reset_index()
    
    # Organização das colunas conforme ordem definido durante a geração do modelo
    colunas = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 
              'quantidade_de_fermento', 'tempo_de_batimento_1ª_fase', 'tempo_de_batimento_2ª_fase', 'temperatura_de_massa',
              'peso_cru', 'altura_da_faca','pressão_rolo_de_borracha_direito', 'pressão_rolo_de_borracha_esquerdo','velocidade_da_moldadora',
              'velocidade_forçador', 'velocidade_lona_moldadora', 'pressão_rolo_estriado',
              'tempo_de_cozimento', 'pressão_do_lastro_-_zona_1', 'pressão_do_lastro_-_zona_2', 'pressão_do_lastro_-_zona_3', 'pressão_do_lastro_-_zona_4',
              'pressão_do_teto_-_zona_1', 'pressão_do_teto_-_zona_2', 'pressão_do_teto_-_zona_3', 'pressão_do_teto_-_zona_4',
              'temperatura_zona_1', 'temperatura_zona_2', 'temperatura_zona_3', 'temperatura_zona_4',
              'comprimento', 'largura', 'peso_assado', 'umidade_(água)']

    table= table.loc[:, colunas]

    table = pd.DataFrame(model_sc_fit.transform(table), columns=colunas) # Aplicando a normalização aos dados inseridos
    return table

  def makePredictions(file, model):
    """Retorna o valor da expansão predito"""
    return model.predict(file)

  # Obtendo Data e hora
  data_e_hora_atuais = datetime.now()
  data_e_hora_em_texto = data_e_hora_atuais.strftime('%d/%m/%Y - %H:%M:%S')

  print(f"Funções carregadas com sucesso! em: {data_e_hora_em_texto} \n\t** Já pode prosseguir para a predição da Expansão **")
  print("##############################################################################")



---

In [ ]:
#@title 2 - Realizar a predição, aperte o  [  ▶ ] 

# Carregamento dos modelos
try:
  model_pkl = '/content/drive/MyDrive/DNC_PredicaoExpansao/pkl_models/model_pkl' # Apontar caminho para pkl_models/model_pkl
  best_model = load_model(model_pkl)[0]   # Modelo com maior R2 obtido gerado (target=expansao)
  lr_model = load_model(model_pkl)[1]     # Modelo de Regressão Linear gerado (target=peso_assado)
except FileNotFoundError:
  print("Favor verificar se o arquivo 'model_pkl' existe na pasta '/pkl_models'")

except NameError:
  print("Verifique se a etapa 1 foi executada com sucesso.")

else:
    # Carregamento dos StandardScale
  try:
    sc_pkl = '/content/drive/MyDrive/DNC_PredicaoExpansao/pkl_models/sc_pkl'      # Apontar caminho para pkl_models/sc_pkl
    model_sc_fit, linear_sc_fit = load_stdScaler(sc_pkl)

  except FileNotFoundError:
    print("Favor verificar se o arquivo 'scpkl' existe na pasta '/pkl_models'")
  
  else:
    file_name = 'predicao.xlsx'

    if file_name not in os.listdir():
      uploaded = files.upload()
      for fn in uploaded.keys():
        print('User uploaded file "{name}" with length {length} bytes'.format(
              name=fn, length=len(uploaded[fn])))

    # Estatísticas do modelo de predição (best_model)
    best_model_stastistics = list(load_model(model_pkl)[2].values())
    
    # Carregamento dos dados
    try:
      # Variáveis de interesse
      model_data = transform_model_data(file_name, model_sc_fit)
      expansao = makePredictions(model_data, best_model)          # Expansão obtida na predição
      precision = round(best_model_stastistics[0][4], 2)*100      # Precisão do modelo
      std = round(best_model_stastistics[0][3],2)                 # Desvio Padrão do modelo

      print(f'\nA expansão prevista para os dados fornecidos é de: {round(expansao[0],2)}mm\n')
      print("############################")
      print(f"Estatísticas da previsão: \n\tPrecisão = {precision}%\n\tDesvio padrão = {std}")
      print("############################")

    except KeyError:
      print("Favor verificar se o arquivo 'predicao.xlsx' está conforme padrão.")
    except:
      print("\nFaltou carregar o arquivo predicao.xlsx. \nFavor carregar o arquivo e executar novamente")

---

In [8]:
#@title 3 - Verificar melhores parâmetros para a linha

def define_lp_variables(lp_data):
    
    lp_data['Etapa'].iloc[39] = 'expansão'     # Alterando peso_assado para expansão
    features = {'Etapa':("coeficiente", "sc_mean", "sc_scale", "Otimo", "LIE", "LSE")}

    # Preenche o dicionário features
    for i, coef, sc_mean, sc_scale in zip(lp_data.index, lr_model.coef_[0], linear_sc_fit.mean_, linear_sc_fit.scale_):
        otimo = lp_data.iloc[i]['Otimo']
        lie = lp_data.iloc[i]['LIE']
        lse = lp_data.iloc[i]['LSE']
        features[lp_data.iloc[i]['Etapa']] = (coef, sc_mean, sc_scale, otimo, lie, lse)

    return features

def create_lp_problem(features, var_expansao, laudo):
    # Create the model
    model = LpProblem(name="Melhores parâmetros", sense=LpMinimize)

    # Initialize the decision variables
    a = LpVariable(name='a', lowBound=features['a'][4], upBound=features['a'][5])
    b = LpVariable(name='b', lowBound=features['b'][4], upBound=features['b'][5])
    c = LpVariable(name='c', lowBound=features['c'][4], upBound=features['c'][5])
    d = LpVariable(name='d', lowBound=features['d'][4], upBound=features['d'][5])
    e = LpVariable(name='e', lowBound=features['e'][4], upBound=features['e'][5])
    f = LpVariable(name='f', lowBound=features['f'][4], upBound=features['f'][5])
    g = LpVariable(name='g', lowBound=features['g'][4], upBound=features['g'][5])
    h = LpVariable(name='h', lowBound=features['h'][4], upBound=features['h'][5])
    i = LpVariable(name='i', lowBound=features['i'][4], upBound=features['i'][5])
    j = LpVariable(name='j', lowBound=features['j'][4], upBound=features['j'][5])
    k = LpVariable(name='k', lowBound=features['k'][4], upBound=features['k'][5])
    l = LpVariable(name='l', lowBound=features['l'][4], upBound=features['l'][5])
    quantidade_de_fermento = LpVariable(name='quantidade_de_fermento', lowBound=features['quantidade_de_fermento'][4], upBound=features['quantidade_de_fermento'][5])
    tempo_de_batimento_1_fase = LpVariable(name='tempo_de_batimento_1ª_fase', lowBound=features['tempo_de_batimento_1ª_fase'][4], upBound=features['tempo_de_batimento_1ª_fase'][5])
    tempo_de_batimento_2_fase = LpVariable(name='tempo_de_batimento_2ª_fase', lowBound=features['tempo_de_batimento_2ª_fase'][4], upBound=features['tempo_de_batimento_2ª_fase'][5])
    temperatura_de_massa = LpVariable(name='temperatura_de_massa', lowBound=features['temperatura_de_massa'][4], upBound=features['temperatura_de_massa'][5])
    peso_cru = LpVariable(name='peso_cru', lowBound=features['peso_cru'][4], upBound=features['peso_cru'][5])
    altura_da_faca = LpVariable(name='altura_da_faca', lowBound=features['altura_da_faca'][4], upBound=features['altura_da_faca'][5])
    pressao_rolo_de_borracha_direito = LpVariable(name='pressão_rolo_de_borracha_direito', lowBound=features['pressão_rolo_de_borracha_direito'][4], upBound=features['pressão_rolo_de_borracha_direito'][5])
    pressao_rolo_de_borracha_esquerdo = LpVariable(name='pressão_rolo_de_borracha_esquerdo', lowBound=features['pressão_rolo_de_borracha_esquerdo'][4], upBound=features['pressão_rolo_de_borracha_esquerdo'][5])
    velocidade_da_moldadora = LpVariable(name='velocidade_da_moldadora', lowBound=features['velocidade_da_moldadora'][4], upBound=features['velocidade_da_moldadora'][5])
    velocidade_forcador = LpVariable(name='velocidade_forçador', lowBound=features['velocidade_forçador'][4], upBound=features['velocidade_forçador'][5])
    velocidade_lona_moldadora = LpVariable(name='velocidade_lona_moldadora', lowBound=features['velocidade_lona_moldadora'][4], upBound=features['velocidade_lona_moldadora'][5])
    pressao_rolo_estriado = LpVariable(name='pressão_rolo_estriado', lowBound=features['pressão_rolo_estriado'][4], upBound=features['pressão_rolo_estriado'][5])
    tempo_de_cozimento = LpVariable(name='tempo_de_cozimento', lowBound=features['tempo_de_cozimento'][4], upBound=features['tempo_de_cozimento'][5])
    pressao_do_lastro_zona_1 = LpVariable(name='pressão_do_lastro_-_zona_1', lowBound=features['pressão_do_lastro_-_zona_1'][4], upBound=features['pressão_do_lastro_-_zona_1'][5])
    pressao_do_lastro_zona_2 = LpVariable(name='pressão_do_lastro_-_zona_2', lowBound=features['pressão_do_lastro_-_zona_2'][4], upBound=features['pressão_do_lastro_-_zona_2'][5])
    pressao_do_lastro_zona_3 = LpVariable(name='pressão_do_lastro_-_zona_3', lowBound=features['pressão_do_lastro_-_zona_3'][4], upBound=features['pressão_do_lastro_-_zona_3'][5])
    pressao_do_lastro_zona_4 = LpVariable(name='pressão_do_lastro_-_zona_4', lowBound=features['pressão_do_lastro_-_zona_4'][4], upBound=features['pressão_do_lastro_-_zona_4'][5])
    pressao_do_teto_zona_1 = LpVariable(name='pressão_do_teto_-_zona_1', lowBound=features['pressão_do_teto_-_zona_1'][4], upBound=features['pressão_do_teto_-_zona_1'][5])
    pressao_do_teto_zona_2 = LpVariable(name='pressão_do_teto_-_zona_2', lowBound=features['pressão_do_teto_-_zona_2'][4], upBound=features['pressão_do_teto_-_zona_2'][5])
    pressao_do_teto_zona_3 = LpVariable(name='pressão_do_teto_-_zona_3', lowBound=features['pressão_do_teto_-_zona_3'][4], upBound=features['pressão_do_teto_-_zona_3'][5])
    pressao_do_teto_zona_4 = LpVariable(name='pressão_do_teto_-_zona_4', lowBound=features['pressão_do_teto_-_zona_4'][4], upBound=features['pressão_do_teto_-_zona_4'][5])
    temperatura_zona_1 = LpVariable(name='temperatura_zona_1', lowBound=features['temperatura_zona_1'][4], upBound=features['temperatura_zona_1'][5])
    temperatura_zona_2 = LpVariable(name='temperatura_zona_2', lowBound=features['temperatura_zona_2'][4], upBound=features['temperatura_zona_2'][5])
    temperatura_zona_3 = LpVariable(name='temperatura_zona_3', lowBound=features['temperatura_zona_3'][4], upBound=features['temperatura_zona_3'][5])
    temperatura_zona_4 = LpVariable(name='temperatura_zona_4', lowBound=features['temperatura_zona_4'][4], upBound=features['temperatura_zona_4'][5])
    comprimento = LpVariable(name='comprimento', lowBound=features['comprimento'][4], upBound=features['comprimento'][5])
    largura = LpVariable(name='largura', lowBound=features['largura'][4], upBound=features['largura'][5])
    expansao = LpVariable(name='expansão')
    expansao_ = LpConstraint(sense=0, name='expansão', rhs=var_expansao)
    umidade = LpVariable(name='umidade_(água)', lowBound=features['umidade_(água)'][4], upBound=features['umidade_(água)'][5])
    peso_assado = LpConstraint(sense= -1, name='peso_assado', rhs=features['expansão'][5])      # Estamos buscando o LSE de expansão, pois é onde está registrado o limite de peso assado no df

    # Define Initial Values
    a.setInitialValue(features['a'][3], check=False)
    b.setInitialValue(features['b'][3], check=False)
    c.setInitialValue(features['c'][3], check=False)
    d.setInitialValue(features['d'][3], check=False)
    e.setInitialValue(features['e'][3], check=False)
    f.setInitialValue(features['f'][3], check=False)
    g.setInitialValue(features['g'][3], check=False)
    h.setInitialValue(features['h'][3], check=False)
    i.setInitialValue(features['i'][3], check=False)
    j.setInitialValue(features['j'][3], check=False)
    k.setInitialValue(features['k'][3], check=False)
    l.setInitialValue(features['l'][3], check=False)
    quantidade_de_fermento.setInitialValue(features['quantidade_de_fermento'][3], check=False)
    tempo_de_batimento_1_fase.setInitialValue(features['tempo_de_batimento_1ª_fase'][3], check=False)
    tempo_de_batimento_2_fase.setInitialValue(features['tempo_de_batimento_2ª_fase'][3], check=False)
    temperatura_de_massa.setInitialValue(features['temperatura_de_massa'][3], check=False)
    peso_cru.setInitialValue(features['peso_cru'][3], check=False)
    altura_da_faca.setInitialValue(features['altura_da_faca'][3], check=False)
    pressao_rolo_de_borracha_direito.setInitialValue(features['pressão_rolo_de_borracha_direito'][3], check=False)
    pressao_rolo_de_borracha_esquerdo.setInitialValue(features['pressão_rolo_de_borracha_esquerdo'][3], check=False)
    velocidade_da_moldadora.setInitialValue(features['velocidade_da_moldadora'][3], check=False)
    velocidade_forcador.setInitialValue(features['velocidade_forçador'][3], check=False)
    velocidade_lona_moldadora.setInitialValue(features['velocidade_lona_moldadora'][3], check=False)
    pressao_rolo_estriado.setInitialValue(features['pressão_rolo_estriado'][3], check=False)
    tempo_de_cozimento.setInitialValue(features['tempo_de_cozimento'][3], check=False)
    pressao_do_lastro_zona_1.setInitialValue(features['pressão_do_lastro_-_zona_1'][3], check=False)
    pressao_do_lastro_zona_2.setInitialValue(features['pressão_do_lastro_-_zona_2'][3], check=False)
    pressao_do_lastro_zona_3.setInitialValue(features['pressão_do_lastro_-_zona_3'][3], check=False)
    pressao_do_lastro_zona_4.setInitialValue(features['pressão_do_lastro_-_zona_4'][3], check=False)
    pressao_do_teto_zona_1.setInitialValue(features['pressão_do_teto_-_zona_1'][3], check=False)
    pressao_do_teto_zona_2.setInitialValue(features['pressão_do_teto_-_zona_2'][3], check=False)
    pressao_do_teto_zona_3.setInitialValue(features['pressão_do_teto_-_zona_3'][3], check=False)
    pressao_do_teto_zona_4.setInitialValue(features['pressão_do_teto_-_zona_4'][3], check=False)
    temperatura_zona_1.setInitialValue(features['temperatura_zona_1'][3], check=False)
    temperatura_zona_2.setInitialValue(features['temperatura_zona_2'][3], check=False)
    temperatura_zona_3.setInitialValue(features['temperatura_zona_3'][3], check=False)
    temperatura_zona_4.setInitialValue(features['temperatura_zona_4'][3], check=False)
    comprimento.setInitialValue(features['comprimento'][3], check=False)
    largura.setInitialValue(features['largura'][3], check=False)
    umidade.setInitialValue(features['umidade_(água)'][3], check=False)

    # Define constraint for linear expression
    model += peso_assado
    model += expansao_ 
    model += largura == comprimento

    # Define linear expression
    model += lpSum([lr_model.intercept_[0],
                    features['a'][0] * ((a - features['a'][1]) / features['a'][2]),
                    features['b'][0] * ((b - features['b'][1]) / features['b'][2]),
                    features['c'][0] * ((c - features['c'][1]) / features['c'][2]),
                    features['d'][0] * ((d - features['d'][1]) / features['d'][2]),
                    features['e'][0] * ((e - features['e'][1]) / features['e'][2]),
                    features['f'][0] * ((f - features['f'][1]) / features['f'][2]),
                    features['g'][0] * ((g - features['g'][1]) / features['g'][2]),
                    features['h'][0] * ((h - features['h'][1]) / features['h'][2]),
                    features['i'][0] * ((i - features['i'][1]) / features['i'][2]),
                    features['j'][0] * ((j - features['j'][1]) / features['j'][2]),
                    features['k'][0] * ((k - features['k'][1]) / features['k'][2]),
                    features['l'][0] * ((l - features['l'][1]) / features['l'][2]),
                    features['quantidade_de_fermento'][0] * ((quantidade_de_fermento - features['quantidade_de_fermento'][1]) / features['quantidade_de_fermento'][2]),
                    features['tempo_de_batimento_1ª_fase'][0] * ((tempo_de_batimento_1_fase - features['tempo_de_batimento_1ª_fase'][1]) / features['tempo_de_batimento_1ª_fase'][2]),
                    features['tempo_de_batimento_2ª_fase'][0] * ((tempo_de_batimento_2_fase - features['tempo_de_batimento_2ª_fase'][1]) / features['tempo_de_batimento_2ª_fase'][2]),
                    features['temperatura_de_massa'][0] * ((temperatura_de_massa - features['temperatura_de_massa'][1]) / features['temperatura_de_massa'][2]),
                    features['peso_cru'][0] * ((peso_cru - features['peso_cru'][1]) / features['peso_cru'][2]),
                    features['altura_da_faca'][0] * ((altura_da_faca - features['altura_da_faca'][1]) / features['altura_da_faca'][2]),
                    features['pressão_rolo_de_borracha_direito'][0] * ((pressao_rolo_de_borracha_direito - features['pressão_rolo_de_borracha_direito'][1]) / features['pressão_rolo_de_borracha_direito'][2]),
                    features['pressão_rolo_de_borracha_esquerdo'][0] * ((pressao_rolo_de_borracha_esquerdo - features['pressão_rolo_de_borracha_esquerdo'][1]) / features['pressão_rolo_de_borracha_esquerdo'][2]),
                    features['velocidade_da_moldadora'][0] * ((velocidade_da_moldadora - features['velocidade_da_moldadora'][1]) / features['velocidade_da_moldadora'][2]),
                    features['velocidade_forçador'][0] * ((velocidade_forcador - features['velocidade_forçador'][1]) / features['velocidade_forçador'][2]),
                    features['velocidade_lona_moldadora'][0] * ((velocidade_lona_moldadora - features['velocidade_lona_moldadora'][1]) / features['velocidade_lona_moldadora'][2]),
                    features['pressão_rolo_estriado'][0] * ((pressao_rolo_estriado - features['pressão_rolo_estriado'][1]) / features['pressão_rolo_estriado'][2]),
                    features['tempo_de_cozimento'][0] * ((tempo_de_cozimento - features['tempo_de_cozimento'][1]) / features['tempo_de_cozimento'][2]),
                    features['pressão_do_lastro_-_zona_1'][0] * ((pressao_do_lastro_zona_1 - features['pressão_do_lastro_-_zona_1'][1]) / features['pressão_do_lastro_-_zona_1'][2]),
                    features['pressão_do_lastro_-_zona_2'][0] * ((pressao_do_lastro_zona_2 - features['pressão_do_lastro_-_zona_2'][1]) / features['pressão_do_lastro_-_zona_2'][2]),
                    features['pressão_do_lastro_-_zona_3'][0] * ((pressao_do_lastro_zona_3 - features['pressão_do_lastro_-_zona_3'][1]) / features['pressão_do_lastro_-_zona_3'][2]),
                    features['pressão_do_lastro_-_zona_4'][0] * ((pressao_do_lastro_zona_4 - features['pressão_do_lastro_-_zona_4'][1]) / features['pressão_do_lastro_-_zona_4'][2]),
                    features['pressão_do_teto_-_zona_1'][0] * ((pressao_do_teto_zona_1 - features['pressão_do_teto_-_zona_1'][1]) / features['pressão_do_teto_-_zona_1'][2]),
                    features['pressão_do_teto_-_zona_2'][0] * ((pressao_do_teto_zona_2 - features['pressão_do_teto_-_zona_2'][1]) / features['pressão_do_teto_-_zona_2'][2]),
                    features['pressão_do_teto_-_zona_3'][0] * ((pressao_do_teto_zona_3 - features['pressão_do_teto_-_zona_3'][1]) / features['pressão_do_teto_-_zona_3'][2]),
                    features['pressão_do_teto_-_zona_4'][0] * ((pressao_do_teto_zona_4 - features['pressão_do_teto_-_zona_4'][1]) / features['pressão_do_teto_-_zona_4'][2]),
                    features['temperatura_zona_1'][0] * ((temperatura_zona_1 - features['temperatura_zona_1'][1]) / features['temperatura_zona_1'][2]),
                    features['temperatura_zona_2'][0] * ((temperatura_zona_2 - features['temperatura_zona_2'][1]) / features['temperatura_zona_2'][2]),
                    features['temperatura_zona_3'][0] * ((temperatura_zona_3 - features['temperatura_zona_3'][1]) / features['temperatura_zona_3'][2]),
                    features['temperatura_zona_4'][0] * ((temperatura_zona_4 - features['temperatura_zona_4'][1]) / features['temperatura_zona_4'][2]),
                    features['comprimento'][0] * ((comprimento - features['comprimento'][1]) / features['comprimento'][2]),
                    features['largura'][0] * ((largura - features['largura'][1]) / features['largura'][2]),
                    features['umidade_(água)'][0] * ((umidade - features['umidade_(água)'][1]) / features['umidade_(água)'][2]),
                    features['expansão'][0] * ((expansao - features['expansão'][1]) / features['expansão'][2])])

    # Verifica se quer utilizar os dados de Laudo informados na planilha
    if laudo == 's':
        model += a == features['a'][0] * ((features['a'][3] - features['a'][1]) / features['a'][2])
        model += b == features['b'][0] * ((features['b'][3] - features['b'][1]) / features['b'][2])
        model += c == features['c'][0] * ((features['c'][3] - features['c'][1]) / features['c'][2])
        model += d == features['d'][0] * ((features['d'][3] - features['d'][1]) / features['d'][2])
        model += e == features['e'][0] * ((features['e'][3] - features['e'][1]) / features['e'][2])
        model += f == features['f'][0] * ((features['f'][3] - features['f'][1]) / features['f'][2])
        model += g == features['g'][0] * ((features['g'][3] - features['g'][1]) / features['g'][2])
        model += h == features['h'][0] * ((features['h'][3] - features['h'][1]) / features['h'][2])
        model += i == features['i'][0] * ((features['i'][3] - features['i'][1]) / features['i'][2])
        model += j == features['j'][0] * ((features['j'][3] - features['j'][1]) / features['j'][2])
        model += k == features['k'][0] * ((features['k'][3] - features['k'][1]) / features['k'][2])
        model += l == features['l'][0] * ((features['l'][3] - features['l'][1]) / features['l'][2])

    return model

def create_lp_model(lp_problem, solver = pulp.PULP_CBC_CMD(msg=True, warmStart=True)):
    status = lp_problem.solve(solver)
    lp_problem.to_dict()

    # print(f"status: {lp_problem.status}, {LpStatus[lp_problem.status]}")

    # print(f"objective: {lp_problem.objective.value()}")

    for var in lp_problem.variables():
        print(f"{var.name}: {var.value()}")

    return

laudo = input("Deseja utilizar os valores de Laudo informados na planilha? Digite:\n \t(s)-sim  -  (n)-não  => ").lower()
var_expansao = float(input("Qual é a expansão esperada para 10 unidades: =>  "))

lp_data = load_file(file_name)
features = define_lp_variables(lp_data)
lp_problem = create_lp_problem(features, var_expansao, laudo)
status = create_lp_model(lp_problem, solver = pulp.PULP_CBC_CMD(msg=True, warmStart=True))

Deseja utilizar os valores de Laudo informados na planilha? Digite:
 	(s)-sim  -  (n)-não  => s
Qual é a expansão esperada para 10 unidades: =>  57.5
a: 0.7
altura_da_faca: 8.0
b: 170.0
c: 0.9
comprimento: 38.0
d: 200.0
e: 57.0
expansão: 0.0
f: 5.0
g: 24.0
h: 8.0
i: 14.5
j: 20.0
k: 65.0
l: 46.4
largura: 38.0
peso_cru: 36.9
pressão_do_lastro___zona_1: 250.0
pressão_do_lastro___zona_2: 500.0
pressão_do_lastro___zona_3: 450.0
pressão_do_lastro___zona_4: 200.0
pressão_do_teto___zona_1: 250.0
pressão_do_teto___zona_2: 500.0
pressão_do_teto___zona_3: 450.0
pressão_do_teto___zona_4: 200.0
pressão_rolo_de_borracha_direito: 230.0
pressão_rolo_de_borracha_esquerdo: 230.0
pressão_rolo_estriado: 8.0
quantidade_de_fermento: 10.0
temperatura_de_massa: 24.0
temperatura_zona_1: 200.0
temperatura_zona_2: 180.0
temperatura_zona_3: 165.0
temperatura_zona_4: 130.0
tempo_de_batimento_1ª_fase: 5.0
tempo_de_batimento_2ª_fase: 1.3
tempo_de_cozimento: 4.3
umidade_(água): 0.8
velocidade_da_moldadora: 14.6
veloc